<a href="https://colab.research.google.com/github/ranaalisaeed/college-student-qa-bot/blob/main/acap_ss_rag_step1_create_n_populate_vdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages:
# jq: JSON processor required by JSON loader
# langchain-community: LangChain community tools
# chromadb: a vector database for embeddings
# tiktoken: required by OpenAIEmbeddings for tokenization
# langchain-chroma: Chroma is deprecated in LangChain 0.2.9

%pip install jq langchain-community langchain-chroma tiktoken --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.0 MB/s eta 0:00:0

In [ ]:
import os
from langchain_community.document_loaders import JSONLoader
from langchain_chroma import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from google.colab import drive, userdata
from pprint import pprint
import shutil

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
loader = JSONLoader(
    file_path="/content/drive/Shareddrives/cnigenai/fees-payments-acap-claude.json",
    jq_schema=".[]",
    text_content=False)

documents = loader.load()

In [ ]:
pprint(documents[:5]) # DEBUG

[Document(metadata={'source': '/content/drive/Shareddrives/cnigenai/fees-payments-acap-claude.json', 'seq_num': 1}, page_content='{"page_title": "Fees and Payments", "page_url": "https://www.acap.edu.au/pages/fees-payments/", "section_title": "Fee Schedule", "section_content": "Our Fee Schedule is a comprehensive guide to course and unit fees. It outlines each ACAP University College course on offer, including information on units, student load and credit points. It also includes fee information for higher education courses. Courses may not be available at all campuses, or in all trimesters. For full information about courses, please refer to course pages .\\n\\n2025 Fee Schedule east Tuition Payment Options east", "section_type": "h2_section", "debug_info": "Found 2 paragraphs"}'),
 Document(metadata={'source': '/content/drive/Shareddrives/cnigenai/fees-payments-acap-claude.json', 'seq_num': 2}, page_content='{"page_title": "Fees and Payments", "page_url": "https://www.acap.edu.au/pag

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

<ipython-input-6-472716ae61dc>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [ ]:
drive.mount('/content/drive')
persist_dir = '/content/drive/MyDrive/acap-db'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

In [ ]:
vectorstore = Chroma.from_documents(documents, embedding=embeddings, persist_directory=persist_dir)

In [ ]:
collection = vectorstore._collection
print("Number of items in the collection: ", collection.count())

Number of items in the collection:  240
